# Implementação Ad-hoc do algoritmo KNN, utilizando medidas de similaridade entre curvas

Este notebook apresenta uma implementação ad-hoc do KNN, utilizando as medidas de similaridade entre curvas DTW e Distância de Fréchet como métrica as  para classificar os sensores de detecção da bactéria E-Coli, desenvolvidos pelo grupo de pesquisa do Programa de Pós-Graduação em Física da UFMG, conforme sua etapa de fabricação, dado uma curva de tensão de entrada e suas respectivas medidas de tensão de saída.

## Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import similaritymeasures
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy import stats

## Leitura dos dados

In [2]:
def load_dataset(df: pd.DataFrame, test_size: float = 0.2):
    X = []
    Y = []

    for label in df['label'].unique():
        for sample in df['amostra'].unique():
            x = df.loc[
                (df['label'] == label) & (df['amostra'] == sample),
                ['v_gate', 'resist']
            ].values
            if len(x) > 0:
                X.append(x)
                Y.append(label)

    return train_test_split(X, Y, test_size=test_size)


## Implementação

In [3]:
class KnnForTimeSeries(object):
    def __init__(self, training_set, training_labels, k=7, metric='dtw'):
        self._training_labels = training_labels
        self._training_set = training_set
        self._k = k
        self._metric = metric

    def _get_neighbors(self, test_instance):
        distances = []
        if self._metric == 'dtw':
            for x in self._training_set:
                distances.append(similaritymeasures.dtw(x, test_instance)[0])
        elif self._metric == 'frechet':
            for x in self._training_set:
                distances.append(similaritymeasures.frechet_dist(x, test_instance))

        neighbors = list(zip(distances, self._training_labels))
        neighbors.sort(key=lambda tup: tup[0])
        return neighbors[:self._k]

    def predict_one(self, test_instance):
        k_nearest = np.vstack(self._get_neighbors(test_instance))
        return stats.mode(k_nearest[:, 1])[0]

    def predict(self, test_set):
        return [self.predict_one(x) for x in test_set]

## Leitura dos dados

In [4]:
split = 0.2
x_train, x_test, y_train, y_test = load_dataset(
    pd.read_csv('all_treated.csv', sep=';'),
    split
)

## Treinamento e apuração dos resultados, utilizando DTW

In [5]:
classifier = KnnForTimeSeries(x_train, y_train)
y_true, y_pred = y_test, classifier.predict(x_test)

target_names = ['Limpo', 'Ácido', 'Molécula']

print(metrics.accuracy_score(y_true, y_pred))
print(metrics.classification_report(y_true, y_pred, target_names=target_names))

0.5919540229885057
              precision    recall  f1-score   support

       Limpo       0.59      0.84      0.70        45
       Ácido       0.66      0.47      0.55        70
    Molécula       0.53      0.54      0.54        59

    accuracy                           0.59       174
   macro avg       0.60      0.62      0.60       174
weighted avg       0.60      0.59      0.58       174



## Treinamento e apuração dos resultadosm, utilizando Fréchet

In [ ]:
classifier_frechet = KnnForTimeSeries(x_train, y_train, metric='frechet')
y_true, y_pred = y_test, classifier_frechet.predict(x_test)

In [7]:
print(metrics.accuracy_score(y_true, y_pred))
print(metrics.classification_report(y_true, y_pred, target_names=target_names))

0.5344827586206896
              precision    recall  f1-score   support

       Limpo       0.50      0.78      0.61        45
       Ácido       0.65      0.40      0.50        70
    Molécula       0.49      0.51      0.50        59

    accuracy                           0.53       174
   macro avg       0.55      0.56      0.53       174
weighted avg       0.56      0.53      0.53       174

